In [1]:
%load_ext line_profiler

In [2]:
from collections import Counter
from functools import partial

import numpy as np
rng = np.random.default_rng()
np.set_printoptions(suppress=True, linewidth=150, edgeitems=5)
from tqdm.notebook import tqdm, trange
from tqdm.contrib.concurrent import process_map

from scipy.stats import rankdata
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [236]:
# ns = [0,10,10,11,12,13,14,15,16,17]
# ns = [0,5,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
# ns = [0]


students = np.logspace(-1, 1, 21)
display(students.round(4))


# ms = 5
# arrs = np.triu(np.tile(np.arange(32, 37),(5,1)).T) - 20
# display(arrs)
# display(arrs.sum())

# ms = 4
# arrs = np.triu(np.tile(np.arange(31, 35),(4,1)).T) - 20
# display(arrs)
# display(arrs.sum())


# ns = [0,10,10,11,12,13,14,15,16,17]
ns = [0,5,5,6,7,8,9,10,11,12,13,14,15]
ms = 2
arrs = np.array([[1,-5],[.7,.3]])
display(arrs)
display(arrs.sum().round(2))

array([ 0.1   ,  0.1259,  0.1585,  0.1995,  0.2512,  0.3162,  0.3981,  0.5012,  0.631 ,  0.7943,  1.    ,  1.2589,  1.5849,  1.9953,  2.5119,
        3.1623,  3.9811,  5.0119,  6.3096,  7.9433, 10.    ])

array([[ 1. , -5. ],
       [ 0.7,  0.3]])

-3.0

In [237]:
ranked = False

In [238]:
def make_model(num_nodes=128, num_dense=4):
    
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
    
    inputs = tf.keras.layers.Input(shape=(ms*ms + ms + 1))
    x = tf.keras.layers.Flatten()(inputs)
    
    for _ in range(num_dense):
        x = tf.keras.layers.Dense(num_nodes, activation='relu')(x)
    
    # output1 = tf.keras.layers.Dense(5, name='Y0')(x)
    # output2 = tf.keras.layers.Dense(5, name='Y1')(x)
    # model = tf.keras.models.Model(inputs=inputs, outputs=[output1, output2])
    
    output1 = tf.keras.layers.Dense(ms, name='Y0')(x)
    output1 = tf.keras.layers.Softmax()(output1)
    model = tf.keras.models.Model(inputs=inputs, outputs=output1)

    
    opt = tf.keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1)
    # opt = tf.keras.optimizers.SGD()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    return model

def softmax(x, b, axis=-1):
    x = x - np.max(x)
    return np.exp(x/b) / np.exp(x/b).sum(axis=axis, keepdims=True)

def make_x():
    x = arrs
    
    rand = np.random.random(arrs.shape)
    rand = (rand - 0.5)
    x = x + rand*0.01
    
    x = rng.permuted(x, axis=-1)
    return x

def eval_model(model, n = 1, disable=True):
    rewards = []
    branch_rewards = []
    for _ in trange(n, disable=disable):
        x = make_x()
        x_tree = np.concatenate([np.eye(ms+1).astype(int), np.tile(x.flatten(), (ms+1,1))], axis=1)
        y_pred = model(x_tree)
        branch_reward = (x_tree[0,ms+1:].reshape(ms,ms) * y_pred[1:].numpy()).sum(axis=1)
        rewards.append(y_pred[0].numpy())
        # reward = (y_pred[0] * branch_reward).numpy().sum()
        # rewards.append(reward)
        branch_rewards.append(branch_reward)
    return np.array(rewards), np.array(branch_rewards)

#ranked
def make_data(b, n=1, disable=True):
    X, Y = [], []
    for _ in trange(n, disable=disable):
        
        if ranked:
            # Ranked Rewards
            x = make_x()
            r1 = softmax(rankdata(x,axis=1), b)
            r1sum = (r1*x).sum(axis=1)
            r0 = softmax(rankdata(r1sum), b)
        
        else:
            ## Rewards Directly
            x = make_x()
            r1 = softmax(x, b)
            r1sum = (r1*x).sum(axis=1)
            r0 = softmax(r1sum, b)
        
        pos = [0] * (ms+1)
        pos[0] = 1
        x0 = pos + list(x.flatten())
        y0 = rng.choice(np.arange(ms), p=r0)
        
        pos = [0] * (ms+1)
        pos[y0+1] = 1
        x1 = pos + list(x.flatten())
        y1 = rng.choice(np.arange(ms), p=r1[y0])

        X.append(x0)
        Y.append(y0)
        X.append(x1)
        Y.append(y1)
    
    X = np.array(X)
    Y = np.array(Y)
    
    return X,Y

In [241]:
%%time
for b in students:
    r0s = []
    r1s = []
    for _ in trange(1000, disable=True):
        x = make_x()
        
        if ranked:
            r0 = (softmax(rankdata(x,axis=1), b) * x).sum(axis=1)
            r1 = (softmax(rankdata(r0), b) * r0).sum()
            r0s.append(r0)
            r1s.append(r1)
            
        else:
            r0 = (softmax(x, b) * x).sum(axis=1)
            r1 = (softmax(r0, b) * r0).sum()
            r0s.append(r0)
            r1s.append(r1)
            
        
    r0s = np.array(r0s)
    r1s = np.array(r1s)
    print(f"{b:.3f}, {r1s.mean():.3f}, {r0s.mean(axis=0).round(2)}, {softmax(r0s.mean(axis=0), b).round(2)}")


0.100, 0.987, [1.   0.69], [0.96 0.04]
0.126, 0.976, [1.   0.68], [0.92 0.08]
0.158, 0.963, [1.   0.67], [0.89 0.11]
0.200, 0.948, [1.   0.65], [0.85 0.15]
0.251, 0.931, [1.   0.63], [0.81 0.19]
0.316, 0.912, [1.   0.61], [0.77 0.23]
0.398, 0.892, [1.   0.59], [0.74 0.26]
0.501, 0.873, [1.   0.58], [0.7 0.3]
0.631, 0.854, [1.   0.56], [0.67 0.33]
0.794, 0.835, [1.   0.55], [0.64 0.36]
1.000, 0.811, [0.99 0.54], [0.61 0.39]
1.259, 0.775, [0.95 0.53], [0.58 0.42]
1.585, 0.714, [0.87 0.53], [0.55 0.45]
1.995, 0.624, [0.72 0.52], [0.52 0.48]
2.512, 0.506, [0.5  0.52], [0.5 0.5]
3.162, 0.372, [0.22 0.51], [0.48 0.52]
3.981, 0.233, [-0.09  0.51], [0.46 0.54]
5.012, 0.098, [-0.39  0.51], [0.46 0.54]
6.310, -0.028, [-0.67  0.51], [0.45 0.55]
7.943, -0.143, [-0.92  0.51], [0.46 0.54]
10.000, -0.245, [-1.13  0.5 ], [0.46 0.54]
CPU times: user 751 ms, sys: 18.1 ms, total: 769 ms
Wall time: 767 ms


In [226]:
def train_model(student):
    verbose = False
    if student == students[-1]:
        print(f'student == {student:.3f}')
        verbose = True
    model = make_model()
    Xtrain, Ytrain = make_data(student, 2**16, disable=not verbose)
    model.fit(Xtrain, Ytrain, verbose=verbose)
    reward = eval_model(model, 10000, disable=not verbose)
    model.save(f'models/starting_{student:.3f}.keras')
    return reward

In [227]:
%%time
rewards = process_map(train_model, students, disable=True, max_workers=24)

student == 10.000


  0%|          | 0/65536 [00:00<?, ?it/s]

4096/4096 [==============================] - 41s 9ms/step - loss: 0.6909 - accuracy: 0.5276


  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 1.18 s, sys: 1.37 s, total: 2.55 s
Wall time: 2min 16s


In [228]:
for r, s in zip(rewards, students):
    
    res = r[0]*r[1]
    res = res.sum(axis=1).mean()
    
    print(f"{s:.3f} {res}")

0.100 0.9848771887487561
0.126 0.9687399681052264
0.158 0.9627900795138153
0.200 0.944949940649149
0.251 0.9417325888552684
0.316 0.900875885117395
0.398 0.8876685359148488
0.501 0.8737062612224469
0.631 0.822976123091378
0.794 0.8071001633653557
1.000 0.7453729527556489
1.259 0.6523550241744233
1.585 0.5729465369834931
1.995 0.4356250612322364
2.512 0.45208748916531594
3.162 0.3385125218530636
3.981 0.2886464011160692
5.012 0.1947473991618349
6.310 0.18522471904161403
7.943 0.13480009231755571
10.000 0.09137675067735519


In [204]:
# a = np.array([[10,0], [9,6]])
a = arrs
b = 10
display(a)
print()
display(softmax(a, b=b))
print()
display((a * softmax(a, b=b)).sum(axis=1))

array([[ 1. , -2. ],
       [ 0.7,  0.3]])

array([[0.57444252, 0.42555748],
       [0.50999867, 0.49000133]])

array([-0.27667245,  0.50399947])

# Graphs

In [ ]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
tf.get_logger().setLevel('ERROR')
model2 = tf.keras.models.load_model(f"models/starting_{0.100:.3f}.keras")
%timeit r0, r1 = eval_model(model2, 1)

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()

    def on_batch_end(self, batch, logs=None):
        r0, r1 = eval_model(self.model, 10)
        r = (r0*r1).sum(axis=1).mean()
        
        
        try:
            self.model.rewards.append(r)
            self.model.accs.append(logs['accuracy'])
        except:
            self.model.rewards = [r]
            self.model.accs = [logs['accuracy']]

#         # Add the output to the logs
#         if logs is not None:
#             logs['reward'] = r

In [ ]:
models = {}

In [ ]:
%%time

with open('tree.log', 'a') as f:
    for student in students:
        
        print(f'student {student:.3f}', file=f, flush=True)

        models[student] = {}

        for teacher in students[::2][3:10]:

            print(f'teacher {teacher:.3f}', file=f, flush=True)
            models[student][teacher] = []
            for i in range(5):
                print(f'model {i}', file=f, flush=True)
                # model = make_model()
                model = tf.keras.models.load_model(f"models/starting_{student:.3f}.keras")

                opt = tf.keras.optimizers.Adam()
                loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
                model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

                X, Y = currs[f'{teacher:.3f}']
                model.fit(X[:2**17], Y[:2**17], callbacks=[CustomCallback()], verbose=False)
                # model.fit(X[:2**17], Y[:2**17], callbacks=[CustomCallback()])

                models[student][teacher].append(model)


        #         break
        #     break
        # break

In [ ]:
for student, s_models in models.items():
    # print(student, list(s_models))
    df = {}
    for teacher, model in s_models.items():
        kernel_size = 20
        kernel = np.ones(kernel_size) / kernel_size
        rewards = np.array([m.rewards for m in model]).mean(axis=0)
        rewards = np.convolve(rewards, kernel, mode='valid')
        df[f"{teacher:.3f}"] = rewards
    df = pd.DataFrame(df)
    
    print(f'Student {student:.3f}')
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        ax = sns.lineplot(df[[c for c in df.columns if float(c) >= student]])
        sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        plt.show()
    print()

In [ ]:
df[[c for c in df.columns if float(c) >= 1]]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    ax = sns.lineplot(df)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
1+1

In [ ]:
ax = sns.lineplot(df[df.columns[4:9]])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
students[::2][4:9]

# Eval Training

In [230]:
def eval_training(student, teacher, verbose=False):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)

    
    if student == -1:
        model2 = make_model()
    else:
        model2 = tf.keras.models.load_model(f"models/starting_{student:.3f}.keras")
    # opt = tf.keras.optimizers.SGD()
    opt = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model2.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    nsum = 0
    
    X, Y = currs[f"{teacher:.3f}"]
    perm = np.random.permutation(len(X))
    X, Y = X[perm], Y[perm]
    
    rewards = []
    for i in ns:
        if i != 0:
            n = 2**i
            # X, Y = make_data(teacher, n, disable=True)
                        
            model2.fit(X[nsum:nsum+n], Y[nsum:nsum+n], verbose=False)
            nsum += n
            
        r0, r1 = eval_model(model2, 10)
        reward = (r0*r1).sum(axis=1).mean()
        rewards.append(reward)
        # print(f"{nsum}    ", end='\r')
    return rewards

In [231]:
def exp(student, verbose=True, n=20):
    # teachers = np.arange(student, 0.40001, 0.02)
    
    teachers = students
    
    t_ = list(teachers) * n
    s_ = [student for _ in t_]
    
    res = process_map(eval_training, s_, t_, max_workers=32, chunksize=1)
    
    return np.array(res)

In [232]:
%%time
ntotal = (2**np.array(ns[1:])).sum()

currs = process_map(make_data, students, [ntotal]*len(students), max_workers=6, chunksize=1)
currs = dict(zip([f"{s:.3f}" for s in students], currs))

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 115 ms, sys: 323 ms, total: 438 ms
Wall time: 27.9 s


In [233]:
%%time
exp(-1, n=1).reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 47.9 ms, sys: 533 ms, total: 581 ms
Wall time: 28.8 s


array([[-0.03, -0.13,  0.02, -0.03,  0.41,  0.32,  0.88,  0.94,  0.99,  0.97,  0.99,  0.98,  0.99],
       [-0.1 , -0.27, -0.05, -0.11,  0.38,  0.25,  0.79,  0.85,  0.96,  0.97,  0.97,  0.97,  0.98],
       [-0.05, -0.16,  0.02,  0.01,  0.44,  0.32,  0.82,  0.93,  0.97,  0.97,  0.96,  0.96,  0.97],
       [ 0.  , -0.06,  0.01,  0.06,  0.31,  0.47,  0.79,  0.88,  0.92,  0.95,  0.94,  0.93,  0.94],
       [-0.04, -0.08,  0.02,  0.04,  0.32,  0.36,  0.79,  0.87,  0.93,  0.93,  0.94,  0.94,  0.92],
       [-0.05, -0.09,  0.01,  0.03,  0.31,  0.44,  0.84,  0.81,  0.89,  0.91,  0.9 ,  0.91,  0.9 ],
       [-0.02, -0.04,  0.09,  0.15,  0.41,  0.59,  0.85,  0.88,  0.88,  0.89,  0.88,  0.88,  0.9 ],
       [-0.03, -0.01,  0.07,  0.12,  0.28,  0.47,  0.68,  0.85,  0.84,  0.86,  0.84,  0.85,  0.87],
       [-0.03,  0.01,  0.06,  0.06,  0.21,  0.36,  0.53,  0.76,  0.8 ,  0.84,  0.82,  0.88,  0.83],
       [ 0.04,  0.11,  0.11,  0.13,  0.3 ,  0.44,  0.57,  0.81,  0.77,  0.8 ,  0.8 ,  0.77,  0.78],


In [234]:
%%time
exp(1, n=5).reshape(-1, len(students), len(ns)).mean(axis=0).round(3)

  0%|          | 0/105 [00:00<?, ?it/s]

CPU times: user 137 ms, sys: 603 ms, total: 740 ms
Wall time: 1min 54s


array([[0.746, 0.76 , 0.774, 0.796, 0.837, 0.91 , 0.989, 0.979, 0.985, 0.989, 0.985, 0.986, 0.985],
       [0.746, 0.76 , 0.775, 0.796, 0.835, 0.906, 0.983, 0.97 , 0.976, 0.969, 0.975, 0.974, 0.971],
       [0.746, 0.76 , 0.775, 0.797, 0.835, 0.905, 0.97 , 0.949, 0.965, 0.963, 0.962, 0.96 , 0.963],
       [0.746, 0.76 , 0.775, 0.795, 0.829, 0.894, 0.954, 0.937, 0.937, 0.948, 0.948, 0.941, 0.944],
       [0.746, 0.76 , 0.774, 0.793, 0.826, 0.89 , 0.937, 0.917, 0.922, 0.938, 0.92 , 0.932, 0.924],
       [0.746, 0.76 , 0.774, 0.793, 0.821, 0.879, 0.91 , 0.896, 0.903, 0.911, 0.911, 0.914, 0.904],
       [0.746, 0.76 , 0.775, 0.795, 0.824, 0.871, 0.897, 0.89 , 0.889, 0.896, 0.882, 0.888, 0.89 ],
       [0.746, 0.76 , 0.774, 0.794, 0.815, 0.844, 0.878, 0.886, 0.864, 0.857, 0.858, 0.875, 0.867],
       [0.746, 0.759, 0.773, 0.793, 0.812, 0.824, 0.828, 0.845, 0.833, 0.836, 0.828, 0.839, 0.831],
       [0.746, 0.759, 0.772, 0.788, 0.797, 0.797, 0.773, 0.803, 0.78 , 0.783, 0.784, 0.792, 0.781],


In [235]:
%%time
exp(10, n=5).reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/105 [00:00<?, ?it/s]

CPU times: user 137 ms, sys: 671 ms, total: 807 ms
Wall time: 1min 54s


array([[0.08, 0.11, 0.1 , 0.12, 0.15, 0.23, 0.63, 0.96, 0.98, 0.98, 0.98, 0.98, 0.99],
       [0.08, 0.11, 0.1 , 0.11, 0.15, 0.23, 0.63, 0.93, 0.97, 0.97, 0.97, 0.97, 0.98],
       [0.08, 0.11, 0.1 , 0.11, 0.15, 0.23, 0.63, 0.9 , 0.94, 0.96, 0.96, 0.96, 0.96],
       [0.08, 0.11, 0.1 , 0.11, 0.15, 0.23, 0.62, 0.87, 0.93, 0.94, 0.94, 0.95, 0.95],
       [0.08, 0.11, 0.1 , 0.11, 0.15, 0.23, 0.6 , 0.83, 0.91, 0.92, 0.93, 0.92, 0.93],
       [0.08, 0.11, 0.1 , 0.11, 0.15, 0.24, 0.59, 0.81, 0.89, 0.89, 0.91, 0.91, 0.92],
       [0.08, 0.11, 0.1 , 0.11, 0.15, 0.24, 0.59, 0.81, 0.87, 0.88, 0.88, 0.89, 0.88],
       [0.08, 0.11, 0.1 , 0.12, 0.15, 0.24, 0.58, 0.81, 0.85, 0.85, 0.86, 0.87, 0.87],
       [0.08, 0.11, 0.1 , 0.12, 0.15, 0.24, 0.56, 0.78, 0.82, 0.83, 0.82, 0.83, 0.84],
       [0.08, 0.11, 0.1 , 0.12, 0.15, 0.24, 0.54, 0.75, 0.76, 0.77, 0.77, 0.79, 0.79],
       [0.08, 0.11, 0.1 , 0.12, 0.15, 0.24, 0.5 , 0.69, 0.69, 0.71, 0.73, 0.7 , 0.74],
       [0.08, 0.11, 0.1 , 0.12, 0.15, 0.23,